In [ ]:
# 엑셀 저장에 유용한 패키지 설치
!pip install xlsxwriter

In [ ]:
# 크롤링에 필요한 패키지 설치
!pip install xmltodict

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import datetime
import requests
import xmltodict
import re
import os
import sys
import urllib.request
import json

In [ ]:
import matplotlib as mpl
from scipy import stats

# 노트북 안에 그래프를 그리기 위해
%matplotlib inline

# 그래프에서 격자로 숫자 범위가 눈에 잘 띄도록 ggplot 스타일을 사용
plt.style.use('ggplot')

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

# 한글 깨짐
plt.rcParams['font.family'] = 'NanumGothic'

In [ ]:
# 실적 데이터 불러오고 컬럼명,타입 설정
features = ['broadcast', 'exp_min', 'm_code', 'p_code', 'p_name', 'p_group', 'unit_price', 'total_price']
dtype_list = ['datetime64', 'float64', 'int32', 'int32', 'object', 'object', 'int32', 'float64']
dtypes =  dict(zip(features, dtype_list))

df = pd.read_excel('./bc/bigcontest_record_train.xlsx', sheet_name = 'rawdata_2019(완)', header=1, dtype=dtypes)
df.columns=features
df['sales'] = round(df['total_price']/df['unit_price'],0)
df.p_name.loc[15751] = df.p_name.loc[15752]

df.shape

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


(38309, 9)

In [ ]:
# 방송일시 컬럼 세분화
df["YEAR"] = df['broadcast'].dt.year
df["MONTH"] = df['broadcast'].dt.month
df["DAY"] = df['broadcast'].dt.day
df["HOUR"] = df['broadcast'].dt.hour
df["MINUTE"] = df['broadcast'].dt.minute
df["SECOND"] = df['broadcast'].dt.second
df["YEAR_DAY"] = df['broadcast'].dt.date
df['DAY_NAME'] = df['broadcast'].dt.day_name()
df['DAY_NUM'] = df['broadcast'].dt.weekday + 1
df['code'] = pd.factorize(df['m_code'])[0]
df['p_group_code'] = pd.factorize(df['p_group'])[0]
df['day_of_week_and_hour'] = (df['DAY_NUM']-1)*24 + df['HOUR']
df['no']= np.arange(0,len(df.index))

df = df.reset_index(drop=True)

In [ ]:
# 날시 데이터 가져오기
weather=pd.read_csv('./bc/main_weather.csv',encoding='CP949')
weather['일시'] = pd.to_datetime(weather['일시'])
weather = weather.drop(weather.columns[:2], axis='columns')
weather = weather.groupby('일시').sum()
weather['일시'] = weather.index
weather.rename(columns={'강수량(mm)':'rain','일시':'broadcast'}, inplace=True)
weather = weather.reset_index(drop=True)
weather = weather[['broadcast','rain']]

df = pd.merge(left=df, right=weather, how='left', on=['broadcast'])
df['rain'] = df['rain'].fillna(method='ffill')
df['rain'] = [1 if s >= 3 else 0 for s in df['rain']]

In [ ]:
df

,broadcast,exp_min,m_code,p_code,p_name,p_group,unit_price,total_price,sales,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,YEAR_DAY,DAY_NAME,DAY_NUM,code,p_group_code,day_of_week_and_hour,no,rain
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,53.0,2019,1,1,6,0,0,2019-01-01,Tuesday,2,0,0,30,0,0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,110.0,2019,1,1,6,0,0,2019-01-01,Tuesday,2,0,0,30,1,0
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,82.0,2019,1,1,6,20,0,2019-01-01,Tuesday,2,0,0,30,2,0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,174.0,2019,1,1,6,20,0,2019-01-01,Tuesday,2,0,0,30,3,0
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,167.0,2019,1,1,6,40,0,2019-01-01,Tuesday,2,0,0,30,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,2020,1,1,0,20,0,2020-01-01,Wednesday,3,714,9,48,38304,0
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,2020,1,1,0,40,0,2020-01-01,Wednesday,3,714,9,48,38305,0
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,2020,1,1,1,0,0,2020-01-01,Wednesday,3,714,9,49,38306,0
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,NaN,2020,1,1,1,20,0,2020-01-01,Wednesday,3,715,9,49,38307,0


In [ ]:
# 연간 주번호 feature 추가
tem_list = []
for i in range(len(df)):
    year = df['YEAR']
    month = df['MONTH']
    day = df['DAY']

    week_number = datetime.date(year[i], month[i], day[i]).isocalendar()[1]
    
    #  20년에 연결되는 12/30, 31일 53으로 표현
    if (month[i] == 12 and day[i] == 31) or (year[i] == 2020):
        week_number = 53
        
    tem_list.append(week_number)

df['52_week'] = tem_list

In [ ]:
# 휴일 feature 추가
month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

total = {}
for i in month:
    service_key = 'JJC5IlOjBjhMwMvARvd2a0D3jVjHuA3DDvF1NZ27DH4pbq5Y1bzUmA0zYt6BOvI562qNa8Z87PlDLZZjYHA3YA%3D%3D'
    service_key_decode = requests.utils.unquote(service_key)

    req_params = {'ServiceKey' : service_key_decode, 'solYear' : '2019', 'solMonth' : i}
    req_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo'

    resp = requests.get(req_url, params =req_params)
    xml_data = resp.text
    dict_data = xmltodict.parse(xml_data)

    if dict_data ['response']['body']['items'] is not None:
        data = dict_data ['response']['body']['items']['item']

        total[i] =  data

month_1 = ['01', '03', '06', '08', '12']

공휴일 = []
for  mon in month:
    if mon in month_1 :
        date = total[mon]['locdate']
        holiday = total[mon]['isHoliday']
        공휴일.append(date)

month_2 = ['02', '05', '09', '10']
for mon in month:
    if mon in month_2 :
        len_mon = len(total[mon])
        for d in range(len_mon):
            date = total[mon][d]['locdate']
            holiday = total[mon][d]['isHoliday']
            공휴일.append(date)
            
공휴일 = pd.to_datetime(공휴일, format='%Y-%m-%d', errors='raise')

def holiday(date):  
    if date in 공휴일 :
        holi = 1
    else:
        holi = 0
    return holi

df['공휴일'] = df['YEAR_DAY'].map(lambda x : holiday(x))

def weekend(day_name):
    if day_name in ['Saturday', 'Sunday']:
        holiday =  1
    else :
        holiday = 0
    return holiday

df['weekend'] = df['DAY_NAME'].map(lambda x : weekend(x))

쉬는날 = []
weekend = df['weekend']
holi = df['공휴일']

for i in range(len(df)):
    if weekend[i] == 1 or holi[i] == 1 :
        쉬는날.append(1)
    else:
        쉬는날.append(0)

del df['weekend']
del df['공휴일']
df['holiday'] = 쉬는날

In [ ]:
df

,broadcast,exp_min,m_code,p_code,p_name,p_group,unit_price,total_price,sales,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,YEAR_DAY,DAY_NAME,DAY_NUM,code,p_group_code,day_of_week_and_hour,no,rain,52_week,holiday
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,53.0,2019,1,1,6,0,0,2019-01-01,Tuesday,2,0,0,30,0,0,1,1
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,110.0,2019,1,1,6,0,0,2019-01-01,Tuesday,2,0,0,30,1,0,1,1
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,82.0,2019,1,1,6,20,0,2019-01-01,Tuesday,2,0,0,30,2,0,1,1
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,174.0,2019,1,1,6,20,0,2019-01-01,Tuesday,2,0,0,30,3,0,1,1
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,167.0,2019,1,1,6,40,0,2019-01-01,Tuesday,2,0,0,30,4,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,2020,1,1,0,20,0,2020-01-01,Wednesday,3,714,9,48,38304,0,53,0
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,2020,1,1,0,40,0,2020-01-01,Wednesday,3,714,9,48,38305,0,53,0
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,2020,1,1,1,0,0,2020-01-01,Wednesday,3,714,9,49,38306,0,53,0
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,NaN,2020,1,1,1,20,0,2020-01-01,Wednesday,3,715,9,49,38307,0,53,0


In [ ]:
# 브랜드 이름, 브랜드 파워 feature 추가
건강기능 =df[df['p_group'] == '건강기능']
생활용품 =df[df['p_group'] == '생활용품']
침구 =df[df['p_group'] == '침구']
주방 =df[df['p_group'] == '주방']
가전 =df[df['p_group'] == '가전']
가구 =df[df['p_group'] == '가구']
이미용 =df[df['p_group'] == '이미용']
농수축 =df[df['p_group'] == '농수축']
잡화 =df[df['p_group'] == '잡화']
속옷= df[df['p_group'] == '속옷']
의류=df[df['p_group'] == '의류']
p_group_name = ['의류', '속옷', '농수축', '이미용', '가전', '가구', '침구', '생활용품', '잡화', '건강기능', '주방']

for group in p_group_name :
    
    if group == '건강기능':
        p_name = 건강기능['p_name']
        
        brand_name_list = []
        for name in p_name:
            brand_name = re.sub('(일시불)|(무이자)|\((.*?)\)|(특집)|(단하루)','', name)
            brand_name = re.sub('[\[\]\"]', " ", str(brand_name))
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]

            brand_name = brand_name.replace('광동제약', '광동')
            brand_name = brand_name.replace('광동', '광동제약')

            brand_name = brand_name.replace('100%리얼착즙석류', '100%리얼')
            brand_name = brand_name.replace('100%리얼', '100%리얼착즙석류')

            brand_name = brand_name.replace('통째로', '통째로 착즙한 루비 석류즙 84포')
            brand_name = brand_name.replace('모나코사놀', '뉴트리원')
            brand_name = brand_name.replace('한삼인순홍삼진7박스', '한삼인')

            brand_name_list.append(brand_name)
        건강기능['brand_name'] = brand_name_list
        
    elif group == '생활용품':
        p_name = 생활용품['p_name']
        brand_name_list = []
        for name in p_name:
            brand_name = re.sub('(\((.*?)\))|(초특가)|(일시불)|(무이자)|(일\))|(무\))|(\[(.*?)\])|(1+1)|(ALL)|(파격가)|(1\+1)|(2019년)','', name)
            brand_name = re.sub('(1세트)|(완벽더블구성)|(국내제작)|(국내제조)|(D\))|(5세트)|(가\))|(KF94)|(기본구성)|(한세트)|(New)','', brand_name)

            brand_name = re.sub('[\[\]]', " ", brand_name)
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]

            brand_name = brand_name.replace('보국미니히터BKH-1083P+BKH-1083P','보국미니히터')
            brand_name = brand_name.replace('보국미니히터BKH-1083P','보국미니히터')
            brand_name = brand_name.replace('까사마루행거', '까사마루')


            brand_name = brand_name.replace('붙이는', '붙이는 따스림')
            brand_name_list.append(brand_name)
        생활용품['brand_name'] = brand_name_list
        
    elif group == '침구':
        p_name = 침구['p_name']
        brand_name_list = []
        for name in p_name:

            brand_name = re.sub('(\(일\))|(\(무\))|(일\))|(무\))','', name)
            brand_name = re.sub('[\[\]]', " ", brand_name)
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]
            brand_name = brand_name.replace('리앤코리아', '리앤')
            brand_name_list.append(brand_name)
        침구['brand_name'] = brand_name_list
        
    elif group == '주방':
        p_name = 주방['p_name']
        brand_name_list = []
        for name in p_name:

            brand_name = re.sub('(일시불)|(무이자)|(\((.*?)\))|(19년)|(2019년형)|([0-9]세트)', '', name)
            brand_name = re.sub('(국내생산)|(국내제조)|(신제품)|(프랑스직수입)|(서장훈의)', '', brand_name)
            brand_name = re.sub('[\[\]]', " ", str(brand_name))
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]

            brand_name = brand_name.replace('쿠쿠전기밥솥', '쿠쿠')
            brand_name = brand_name.replace('쿠첸압력밥솥', '쿠첸')
            brand_name = brand_name.replace('쿠첸인버터전자레인지', '쿠첸')

            brand_name = brand_name.replace('해피콜IH양면팬', '해피콜')
            brand_name = brand_name.replace('해피콜엑슬림블렌더', '해피콜')  
            brand_name = brand_name.replace('휴롬퀵스퀴저', '휴롬')
            brand_name_list.append(brand_name)
        주방['brand_name'] = brand_name_list
        
    elif group == '가전':
        p_name = 가전['p_name']
        brand_name_list = []
        for name in p_name:

            brand_name = re.findall('(LG)|(삼성)|(대우전자)|(캐리어)|(딤채)', name)
            brand_name = re.sub('[\'\'\,\[\]\(\)]', '', str(brand_name))
            brand_name = brand_name.strip()
            brand_name = brand_name.replace('삼성    LG', 'LG')
            brand_name_list.append(brand_name)
        가전['brand_name'] = brand_name_list
        
    elif group == '가구':
        p_name = 가구['p_name']
        brand_name_list = []
        for name in p_name:

            brand_name = re.sub('(\(일\))|(\(무\))|(일\))|(무\))|(\[SET\])|(뉴)|(\(무\)[0-9]인용)|([0-9]인용)','', name)
            brand_name = re.sub('[\[\]]', " ", brand_name)
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]
            brand_name_list.append(brand_name)
        가구['brand_name'] = brand_name_list
        
    elif group == '이미용':
        p_name = 이미용['p_name']
        brand_name_list = []
        for name in p_name:

            brand_name = re.sub('(\(무이자\))|(NEW프리미엄)|(\(일시불\))|(\[라쉬반\］)|(\[완벽더블\])|(초특가\))','', name)
            brand_name = re.sub('[\[\]]', " ", brand_name)
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]
            brand_name = brand_name.replace('TS샴푸/트리트먼트(추가)', 'TS')
            brand_name = brand_name.replace('TS샴푸세트(추가)', 'TS')   
            brand_name_list.append(brand_name)
        이미용['brand_name'] = brand_name_list
        
    elif group == '농수축':
        p_name = 농수축['p_name']
        brand_name_list = []
        for name in p_name:

            brand_name = re.sub('(\(무이자\))|(NEW프리미엄)|(\(일시불\))|(\[라쉬반\))|(특등급)|(\[완벽더블\])|(초특가\)|(\(a4\))|(가격인하)|(19년)|(사용불가))','', name)
            brand_name = re.sub('[\[\]]', " ", brand_name)
            brand_name = brand_name.replace('하림뼈없는양념닭발세트', '하림')
            brand_name = brand_name.replace('AAC한우소머리곰탕양곰탕세트', 'AAC')
            brand_name = brand_name.replace('맛있는', '맛있는제주')
            brand_name = brand_name.replace('맛있는', '맛있는제주')
            brand_name = brand_name.replace('더커진거창사과', '거창사과')
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]
            brand_name_list.append(brand_name)
        농수축['brand_name'] = brand_name_list
        
    elif group == '잡화':
        p_name = 잡화['p_name']
        brand_name_list = []
        for name in p_name:

            brand_name = re.sub('(\(무\))|(\(일\))|(무이자)|(일시불)|(2019)|(S/S)', '', name)
            brand_name = re.sub('[\[\]]', " ", brand_name)
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]
            brand_name_list.append(brand_name)
        잡화['brand_name'] = brand_name_list
        
    elif group == '속옷':
        p_name = 속옷['p_name']
        brand_name_list = []
        for name in p_name:
            brand_name = re.sub('[\[\]\(\)］]', " ", name)
            brand_name = re.sub('(ARS10%)|(무이자)|(세일20%)|(일시불)|(완벽더블)|(초특가)', '', brand_name)
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]
            brand_name_list.append(brand_name)
        속옷['brand_name'] = brand_name_list
        
    elif group == '의류':
        p_name = 의류['p_name']
        brand_name_list = []
        for name in p_name:
            brand_name = re.sub('(화이트라벨)|(19 SS)|([0-9])|([\[\]]|(F/W)|(FW))', '', str(name))
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]
            brand_name_list.append(brand_name)
        의류['brand_name'] = brand_name_list
    
    else:
        break
        
p_group_name = [의류, 속옷, 농수축, 이미용, 가전, 가구, 침구, 생활용품, 잡화, 건강기능, 주방]

for c in p_group_name:
    #c_data =data[data['p_group'] == c]
    
    c_grouped = c.groupby('brand_name').sum()
    c_grouped['1분당 판매개수'] = c_grouped['sales'] / c_grouped['exp_min']
    c_dict = c_grouped[['1분당 판매개수']].sort_values('1분당 판매개수', ascending=False).to_dict()
    
    
    p_name = c['brand_name']
    c['brand_power'] = [c_dict['1분당 판매개수'][i] for i in p_name]

result_all = pd.concat([의류, 속옷, 농수축, 이미용, 가전, 가구, 침구, 생활용품, 잡화, 건강기능, 주방])
df = result_all.sort_values('broadcast')
df = df.reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [ ]:
# 마스크, 택배 검색량의 비율 feature 추가 (구간별 검색량의 상대적 비율. 구간별 결과에서 가장 큰 값을 100으로 설정한 상댓값입니다.)
client_id = "EDXLJguStAzjmnKluA9U"
client_secret = "u2Q9XCNqEy"
url = "https://openapi.naver.com/v1/datalab/search"
body = "{\"startDate\":\"2019-01-01\",\"endDate\":\"2020-07-01\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\"마스크\",\"keywords\":[\"마스크\",\"공적마스크\",\"kf-94\",\"kf-80\",\"kf-ad\"]},{\"groupName\":\"택배\",\"keywords\":[\"택배\",\"택배조회\",\"cj대한통운\",\"우체국택배\"]}]}"

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    resp=response_body.decode('utf-8')
else:
    print("Error Code:" + rescode)

naver=json.loads(resp)

mask = naver['results'][0]
mask_data = mask['data']

mask_period_list = []
mask_ratio_list = []


for i in range(len(mask_data)):
    mask_period_list.append(mask_data[i]['period'])
    mask_ratio_list.append(mask_data[i]['ratio'])
    
    
mask_df = pd.DataFrame({'period' : mask_period_list, 'ratio' : mask_ratio_list})

parcel = naver['results'][-1]
parcel_data = parcel['data']

parcel_period_list = []
parcel_ratio_list = []

for i in range(len(parcel_data)):
    parcel_period_list.append(parcel_data[i]['period'])
    parcel_ratio_list.append(parcel_data[i]['ratio'])
    
parcel_df = pd.DataFrame({'period' : parcel_period_list, 'ratio' : parcel_ratio_list})

mask_data=mask['data']
mask_data_value=mask_data[0].values()

period_list=[]
ratio_list=[]


for i in range(len(mask_data)):
    period_list.append(mask_data[i]['period'])
    ratio_list.append(mask_data[i]['ratio'])

mask_df=pd.DataFrame({'period':period_list, 'ratio':ratio_list})

df['YEAR_DAY'] = pd.to_datetime(df['YEAR_DAY'], format='%Y-%m-%d', errors='raise')

mask_df.columns = ['YEAR_DAY','mask_ratio']
mask_df['YEAR_DAY'] = pd.to_datetime(mask_df['YEAR_DAY'], format='%Y-%m-%d', errors='raise')
df = pd.merge(left=df, right=mask_df, how='left', on=['YEAR_DAY'])

parcel_df.columns = ['YEAR_DAY','parcel_ratio']
parcel_df['YEAR_DAY'] = pd.to_datetime(parcel_df['YEAR_DAY'], format='%Y-%m-%d', errors='raise')
df = pd.merge(left=df, right=parcel_df, how='left', on=['YEAR_DAY'])

In [ ]:
# 데이터에서 24시 부터 26시 까지의 데이터를 하루 전날의 실적으로 계산하기 위해서 날짜를 -1 하는 데이터와 원래 데이터로 나눔
df2 = df.copy()
df2 = df2.reset_index(drop=True)

In [ ]:
# 익일 0시부터 2시까지 데이터는 전날 데이터로 계산
hour = df2['broadcast'].dt.hour

df2 = df2.reset_index(drop=True)
for i in range(len(df)) :
  
  if hour[i] ==0 or hour[i] ==1 or hour[i] ==2:
    df2['broadcast'][i] = pd.DatetimeIndex(df2['broadcast'])[i] - timedelta(days=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# 추가된 컬럼들 다시 최신화
df2["YEAR"] = df2['broadcast'].dt.year
df2["MONTH"] = df2['broadcast'].dt.month
df2["DAY"] = df2['broadcast'].dt.day
df2["HOUR"] = df2['broadcast'].dt.hour
df2["MINUTE"] = df2['broadcast'].dt.minute
df2["SECOND"] = df2['broadcast'].dt.second
df2["YEAR_DAY"] = df2['broadcast'].dt.date
df2['DAY_NAME'] = df2['broadcast'].dt.day_name()
df2['DAY_NUM'] = df2['broadcast'].dt.weekday + 1
df2['code'] = pd.factorize(df2['m_code'])[0]
df['p_group_code'] = pd.factorize(df['p_group'])[0]
df2['no']= np.arange(0,len(df2.index))

df2 = df2.reset_index(drop=True)

df2.tail(5)

,broadcast,exp_min,m_code,p_code,p_name,p_group,unit_price,total_price,sales,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,YEAR_DAY,DAY_NAME,DAY_NUM,code,p_group_code,day_of_week_and_hour,no,rain,52_week,holiday,brand_name,brand_power,mask_ratio,parcel_ratio
37367,2019-12-31 23:40:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,69.0,2019,12,31,23,40,0,2019-12-31,Tuesday,2,22,2,47,37367,0,53,0,쿠첸,13.968137,0.03906,1.28643
37368,2019-12-31 00:00:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,87.0,2019,12,31,0,0,0,2019-12-31,Tuesday,2,22,2,48,37368,0,53,0,쿠첸,13.968137,0.03949,0.56837
37369,2019-12-31 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,286.0,2019,12,31,0,0,0,2019-12-31,Tuesday,2,22,2,48,37369,0,53,0,쿠첸,13.968137,0.03949,0.56837
37370,2019-12-31 00:00:00,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,621.0,2019,12,31,0,0,0,2019-12-31,Tuesday,2,22,2,48,37370,0,53,0,쿠첸,13.968137,0.03949,0.56837
37371,2019-12-31 00:00:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,46608000.0,315.0,2019,12,31,0,0,0,2019-12-31,Tuesday,2,22,2,48,37371,0,53,0,쿠첸,13.968137,0.03949,0.56837


In [ ]:
# 인덱스 정리
df = df.reset_index(drop=True)
df2 = df2.reset_index(drop=True)

In [ ]:
# quarter, timeslot, season feature 추가
df2['HOUR'] = df2['HOUR'].replace(0,24).replace(1,25).replace(2,26)

df2['quarter'] = pd.to_datetime(df.broadcast).dt.quarter
df2['timeslot'] = pd.cut(df2.HOUR, bins = [0,8,11,14,17,20,23,26], labels = [1,2,3,4,5,6,7])
seasons = [4, 4, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4]
month_to_season = dict(zip(range(1,13), seasons))
df2['season'] = df2.MONTH.map(month_to_season)

df2['HOUR'] = df2['HOUR'].replace(24,0).replace(25,1).replace(26,2)

In [ ]:
# 무형 데이터와 total_price 가 0 인 데이터 제거
df = df[df.p_group != '무형']
df = df.dropna(subset=['total_price'])
df = df.reset_index(drop=True)

In [ ]:
# 무형 데이터와 total_price 가 0 인 데이터 제거
df2 = df2[df2.p_group != '무형']
df2 = df2.dropna(subset=['total_price'])

df2 = df2.reset_index(drop=True)

In [ ]:
df2

,broadcast,exp_min,m_code,p_code,p_name,p_group,unit_price,total_price,sales,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,YEAR_DAY,DAY_NAME,DAY_NUM,code,p_group_code,day_of_week_and_hour,no,rain,52_week,holiday,brand_name,brand_power,mask_ratio,parcel_ratio,quarter,timeslot,season
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,53.0,2019,1,1,6,0,0,2019-01-01,Tuesday,2,0,0,30,0,0,1,1,테이트,16.386845,0.04095,0.52582,1,1,4
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,110.0,2019,1,1,6,0,0,2019-01-01,Tuesday,2,0,0,30,1,0,1,1,테이트,16.386845,0.04095,0.52582,1,1,4
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,82.0,2019,1,1,6,20,0,2019-01-01,Tuesday,2,0,0,30,2,0,1,1,테이트,16.386845,0.04095,0.52582,1,1,4
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,174.0,2019,1,1,6,20,0,2019-01-01,Tuesday,2,0,0,30,3,0,1,1,테이트,16.386845,0.04095,0.52582,1,1,4
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,167.0,2019,1,1,6,40,0,2019-01-01,Tuesday,2,0,0,30,4,0,1,1,테이트,16.386845,0.04095,0.52582,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,69.0,2019,12,31,23,40,0,2019-12-31,Tuesday,2,22,2,47,37367,0,53,0,쿠첸,13.968137,0.03906,1.28643,4,6,4
35375,2019-12-31 00:00:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,87.0,2019,12,31,0,0,0,2019-12-31,Tuesday,2,22,2,48,37368,0,53,0,쿠첸,13.968137,0.03949,0.56837,1,7,4
35376,2019-12-31 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,286.0,2019,12,31,0,0,0,2019-12-31,Tuesday,2,22,2,48,37369,0,53,0,쿠첸,13.968137,0.03949,0.56837,1,7,4
35377,2019-12-31 00:00:00,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,621.0,2019,12,31,0,0,0,2019-12-31,Tuesday,2,22,2,48,37370,0,53,0,쿠첸,13.968137,0.03949,0.56837,1,7,4


In [ ]:
# 파일을 시트별로 나누어서 저장한다.
writer = pd.ExcelWriter('./bc/base_line.xlsx', engine='xlsxwriter')
df2.to_excel(writer, sheet_name='changed_data', index = False)
df.to_excel(writer, sheet_name='original_data', index = False)
writer.save()

In [ ]:
# 평가 데이터 불러오고 컬럼명,타입 설정 (모델에 넣고 돌려보기 위해 실적데이터와 같은 형태로 만들어줌)
features = ['broadcast', 'exp_min', 'm_code', 'p_code', 'p_name', 'p_group', 'unit_price', 'total_price']
dtype_list = ['datetime64', 'float64', 'int32', 'int32', 'object', 'object', 'int32', 'float64']
dtypes =  dict(zip(features, dtype_list))

df_e = pd.read_excel('./bc/bigcontest_record_eval.xlsx', sheet_name = '6월편성', header=1, dtype=dtypes)
df_e.columns=features

df_e.shape

(2891, 8)

In [ ]:
df_e

,broadcast,exp_min,m_code,p_code,p_name,p_group,unit_price,total_price
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN
...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN


In [ ]:
# 방송일시 컬럼 세분화
df_e["YEAR"] = df_e['broadcast'].dt.year
df_e["MONTH"] = df_e['broadcast'].dt.month
df_e["DAY"] = df_e['broadcast'].dt.day
df_e["HOUR"] = df_e['broadcast'].dt.hour
df_e["MINUTE"] = df_e['broadcast'].dt.minute
df_e["SECOND"] = df_e['broadcast'].dt.second
df_e["YEAR_DAY"] = df_e['broadcast'].dt.date
df_e['DAY_NAME'] = df_e['broadcast'].dt.day_name()
df_e['DAY_NUM'] = df_e['broadcast'].dt.weekday + 1
df_e['code'] = pd.factorize(df_e['m_code'])[0]
df_e['p_group_code'] = pd.factorize(df_e['p_group'])[0]
df_e['day_of_week_and_hour'] = (df_e['DAY_NUM']-1)*24 + df_e['HOUR']
df_e['no']= np.arange(0,len(df_e.index))
df_e['52_week	'] = df_e['broadcast'].dt.weekofyear

df_e = df_e.reset_index(drop=True)

In [ ]:
# 휴일 feature 추가
def holiday(date):  
    if date in 공휴일 :
        holi = 1
    else:
        holi = 0
    return holi

df_e['공휴일'] = df_e['YEAR_DAY'].map(lambda x : holiday(x))

def weekend(day_name):
    if day_name in ['Saturday', 'Sunday']:
        holiday =  1
    else :
        holiday = 0
    return holiday

df_e['weekend'] = df_e['DAY_NAME'].map(lambda x : weekend(x))

쉬는날 = []
weekend = df_e['weekend']
holi = df_e['공휴일']

for i in range(len(df_e)):
    if weekend[i] == 1 or holi[i] == 1 :
        쉬는날.append(1)
    else:
        쉬는날.append(0)

del df_e['weekend']
del df_e['공휴일']
df_e['holiday'] = 쉬는날

In [ ]:
# quarter, timeslot, season feature 추가
df_e['HOUR'] = df_e['HOUR'].replace(0,24).replace(1,25).replace(2,26)

df_e['quarter'] = 2
df_e['timeslot'] = pd.cut(df_e.HOUR, bins = [0,8,11,14,17,20,23,26], labels = [1,2,3,4,5,6,7])
seasons = [4, 4, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4]
month_to_season = dict(zip(range(1,13), seasons))
df_e['season'] = df_e.MONTH.map(month_to_season)

df_e['HOUR'] = df_e['HOUR'].replace(24,0).replace(25,1).replace(26,2)

In [ ]:
# 날씨 feature 추가 (20년 6월의 데이터도 가지고있는 csv 파일을 사용한다.)
weather2=pd.read_csv('./bc/6_main_weather.csv',encoding='CP949')
weather2['일시'] = pd.to_datetime(weather2['일시'])
weather2 = weather2.drop(weather2.columns[:2], axis='columns')
weather2 = weather2.groupby('일시').sum()
weather2['일시'] = weather2.index
weather2.rename(columns={'강수량(mm)':'rain','일시':'broadcast'}, inplace=True)
weather2 = weather2.reset_index(drop=True)
weather2 = weather2[['broadcast','rain']]

df_e = pd.merge(left=df_e, right=weather2, how='left', on=['broadcast'])
df_e['rain'] = df['rain'].fillna(method='ffill')
df_e['rain'] = [1 if s >= 3 else 0 for s in df_e['rain']]

In [ ]:
# 마스크, 택배 검색량의 비율 feature 추가
df_e['YEAR_DAY'] = pd.to_datetime(df_e['YEAR_DAY'], format='%Y-%m-%d', errors='raise')

df_e= pd.merge(left=df_e, right=mask_df, how='left', on=['YEAR_DAY'])
df_e= pd.merge(left=df_e, right=parcel_df, how='left', on=['YEAR_DAY'])

In [ ]:
df_e

,broadcast,exp_min,m_code,p_code,p_name,p_group,unit_price,total_price,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,YEAR_DAY,DAY_NAME,DAY_NUM,code,p_group_code,day_of_week_and_hour,no,52_week\t,holiday,quarter,timeslot,season,rain,mask_ratio,parcel_ratio
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,2020,6,1,6,20,0,2020-06-01,Monday,1,0,0,6,0,23,0,2,1,2,0,1.85848,1.15168
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,2020,6,1,6,40,0,2020-06-01,Monday,1,0,0,6,1,23,0,2,1,2,0,1.85848,1.15168
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,2020,6,1,7,0,0,2020-06-01,Monday,1,0,0,7,2,23,0,2,1,2,0,1.85848,1.15168
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,2020,6,1,7,20,0,2020-06-01,Monday,1,1,1,7,3,23,0,2,1,2,0,1.85848,1.15168
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,2020,6,1,7,40,0,2020-06-01,Monday,1,1,1,7,4,23,0,2,1,2,0,1.85848,1.15168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,2020,7,1,0,20,0,2020-07-01,Wednesday,3,174,5,48,2886,27,0,2,7,2,0,1.11449,1.81406
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,2020,7,1,0,40,0,2020-07-01,Wednesday,3,174,5,48,2887,27,0,2,7,2,0,1.11449,1.81406
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,2020,7,1,1,0,0,2020-07-01,Wednesday,3,174,5,49,2888,27,0,2,7,2,0,1.11449,1.81406
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN,2020,7,1,1,20,0,2020-07-01,Wednesday,3,224,0,49,2889,27,0,2,7,2,0,1.11449,1.81406


In [ ]:
# 익일 0시부터 2시까지 데이터는 전날 데이터로 계산
hour = df_e['broadcast'].dt.hour

df_e = df_e.reset_index(drop=True)
for i in range(len(df_e)) :
  # 익일 0시부터 2시까지 데이터는 전날 데이터로 계산
  if hour[i] ==0 or hour[i] ==1 or hour[i] ==2:
    df_e['broadcast'][i] = pd.DatetimeIndex(df_e['broadcast'])[i] - timedelta(days=1)

In [ ]:
# 날짜 관련 feature 최신화
df_e["MONTH"] = df_e['broadcast'].dt.month
df_e["DAY"] = df_e['broadcast'].dt.day
df_e["YEAR_DAY"] = df_e['broadcast'].dt.date
df_e['DAY_NAME'] = df_e['broadcast'].dt.day_name()
df_e['DAY_NUM'] = df_e['broadcast'].dt.weekday + 1

In [ ]:
# 파일 저장
writer = pd.ExcelWriter('./bc/base_line_eval.xlsx', engine='xlsxwriter')
df_e.to_excel(writer, sheet_name='changed_data', index = False)
writer.save()